In [ ]:
# Install necessary packages
!pip install --quiet --upgrade langchain-text-splitters langchain-community whoosh sentence-transformers pandas
!pip install langchain langchain-milvus pymilvus requests beautifulsoup4 nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.9/275.9 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [ ]:
!pip install langchain-huggingface

In [ ]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=faa1c00af8413c1d5a1136a363a933d76217b1eb4ce6de6b1885fa7278dab8ca
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [ ]:
import os
import requests
import nltk
import pandas as pd
from bs4 import BeautifulSoup
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_milvus import Milvus
from pymilvus import utility, Collection
from langchain_huggingface import HuggingFaceEndpoint, HuggingFaceEmbeddings
from sentence_transformers import CrossEncoder
from whoosh.index import create_in
from whoosh.fields import Schema, TEXT
from whoosh.qparser import QueryParser
from difflib import SequenceMatcher
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
os.environ["LANGCHAIN_API_KEY"] = "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"

In [ ]:
llm = HuggingFaceEndpoint(repo_id="mistralai/Mistral-7B-Instruct-v0.3", model_kwargs={"max_length": 128})
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")
# cross_encoder = CrossEncoder("cross_encoder = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-12-v2")  # MultiBERT variant for better ranking
cross_encoder = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-12-v2")  # MultiBERT variant for better ranking

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/791 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
resources = [
    "https://prometheus.io/docs/prometheus/latest/querying/examples/",
    "https://promlabs.com/promql-cheat-sheet/",
    "https://prometheus.io/docs/prometheus/latest/querying/basics/",
    "https://prometheus.io/docs/prometheus/latest/querying/operators/",
    "https://prometheus.io/docs/prometheus/latest/querying/functions/",
    "https://prometheus.io/docs/prometheus/latest/querying/api/",
]

In [ ]:
def fetch_website_content(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    text = ' '.join(p.get_text() for p in soup.find_all('p'))
    return text

documents = [fetch_website_content(url) for url in resources]

# Load CSV Dataset
csv_path = "/mnt/data/promql_test_dataset.csv"  # Ensure correct path
if os.path.exists(csv_path):
    df = pd.read_csv(csv_path)
    for _, row in df.iterrows():
        documents.append(f"Query: {row['instruction']}\nPromQL: {row['output']}")

In [ ]:
# Split Documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.create_documents(documents)


In [ ]:
# Milvus Lite Setup
URI = "./milvus_promql.db"
collection_name = "prometheus_docs"

from langchain_milvus import Milvus

vector_store = Milvus.from_documents(
    documents=splits,
    embedding=HuggingFaceEmbeddings(model_name="BAAI/bge-m3"),
    collection_name=collection_name,
    connection_args={"uri": URI},
    drop_old=True,
    index_params={"index_type": "IVF_FLAT", "metric_type": "L2"}
)

In [ ]:
# Whoosh BM25 Search
schema = Schema(content=TEXT(stored=True))
index_dir = "whoosh_index"
if not os.path.exists(index_dir):
    os.mkdir(index_dir)
index = create_in(index_dir, schema)
writer = index.writer()
for doc in splits:
    writer.add_document(content=doc.page_content)
writer.commit()

# Retrieval with BM25 + Vectors
class HybridRetriever:
    def __init__(self, vectorstore, index):
        self.vectorstore = vectorstore
        self.index = index

    def get_relevant_documents(self, query):
        vector_results = self.vectorstore.similarity_search(query, k=5)
        parser = QueryParser("content", self.index.schema)
        with self.index.searcher() as searcher:
            bm25_results = searcher.search(parser.parse(query), limit=5)
        bm25_docs = [doc["content"] for doc in bm25_results]
        all_docs = list(set([doc.page_content for doc in vector_results] + bm25_docs))
        return all_docs

retriever = HybridRetriever(vector_store, index)

In [ ]:
prompt_template = """
You are an expert in Prometheus and PromQL.
Given the following documentation:
{context}
Generate the most accurate PromQL query for:
{question}
Please provide only the query, with no additional explanation, context, or comments. The query should be enclosed in triple backticks (```).
"""


def format_docs(docs):
    return " ".join(docs)

def generate_query(context, question):
    prompt = prompt_template.format(context=context, question=question)
    response = llm.invoke(prompt)
    return response.strip()

def rerank_docs(docs, query):
    scores = cross_encoder.predict([(query, doc) for doc in docs])
    return [doc for _, doc in sorted(zip(scores, docs), reverse=True)]

def ask_question(query):
    retrieved_docs = retriever.get_relevant_documents(query)
    ranked_docs = rerank_docs(retrieved_docs, query)
    formatted_context = format_docs(ranked_docs[:3])
    generated_query = generate_query(formatted_context, query)
    print(f"Query: {query}\nGenerated Query: {generated_query}\n")
    return generated_query

In [ ]:
# Evaluation Metrics
def levenshtein_similarity(a, b):
    return SequenceMatcher(None, a, b).ratio()

def evaluate(generated, expected):
    bleu = sentence_bleu([expected.split()], generated.split(), smoothing_function=SmoothingFunction().method1)
    rouge = rouge_scorer.RougeScorer(["rougeL"], use_stemmer=True).score(expected, generated)["rougeL"].fmeasure
    levenshtein = levenshtein_similarity(generated, expected)
    return {"BLEU": bleu, "ROUGE-L": rouge, "Levenshtein": levenshtein}

In [ ]:
# Example Query
test_query = "Get the CPU usage percentage over the last hour."
# expected_output = "avg(rate(node_cpu_seconds_total[1h])) * 100"
generated_output = ask_question(test_query)
# print(evaluate(generated_output, expected_output))


Query: Get the CPU usage percentage over the last hour.
Generated Query: ```
irate(sum by (job, instance) (100 * (rate(container_cpu_usage_seconds_total{job, instance}[1h])) / container_cpu_usage_seconds_total{job, instance}[1h]))
```

